In [1]:
import googleapiclient.discovery
import google_auth_oauthlib.flow
from keras.models import load_model
import os 
os.chdir('..\data')
loaded_model = load_model('best_model.h5')

In [2]:
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
CLIENT_SECRET_FILE = 'client_secret2.json'

In [3]:
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly', 'https://www.googleapis.com/auth/youtube.force-ssl']

# API 클라이언트 생성 및 인증하여 Credential 객체 얻기
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=29898966518-ra5akcd4ltrohgh0igfsi5dkmc87g4a7.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=5Z878c1lHbv5bswuIof9Nu2TqoUFrd&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g5sIbm0qHwTPV4srxE5VJ6zJfDZD4u0LchHSHod9GPyh1qcxPSkMnY


In [4]:
#인증된 사용자 채널의 기본 정보 가져오기
channels_response = youtube.channels().list(
    mine=True,
    part='contentDetails'
).execute()

channel = channels_response['items'][0] # 첫 번째 채널 선택
uploads_playlist_id = channel['contentDetails']['relatedPlaylists']['uploads'] # 업로드 영상 플레이 리스트 ID 추출

print('업로드한 영상의 Playlist ID:', uploads_playlist_id)

업로드한 영상의 Playlist ID: UU-UsPf2NsLI5eNa_gtblmjw


In [11]:
channels_response = youtube.channels().list(
    mine=True,
    part='contentDetails'
)

In [15]:
# channels_response.execute()

In [5]:
# 인증된 사용자 채널의 모든 비디오 정보 가져오기
playlistitems_list_request = youtube.playlistItems().list(
    playlistId=uploads_playlist_id,
    part='snippet',
    maxResults=50
)

cnt = 0
last = 20 # 최근 몇 개까지의 동영상을 확인할지 설정
video_list = []

# 해당 플레이 리스트의 모든 동영상을 하나씩 확인하며
while playlistitems_list_request:
    playlistitems_list_response = playlistitems_list_request.execute()

    # 각 비디오(video)에 대한 정보 출력
    for playlist_item in playlistitems_list_response['items']:
        video_id = playlist_item['snippet']['resourceId']['videoId']
        title = playlist_item['snippet']['title']
        video_list.append((video_id, title))
        cnt += 1
        if cnt >= last:
            break

    if cnt >= last:
        break
    playlistitems_list_request = youtube.playlistItems().list_next(playlistitems_list_request, playlistitems_list_response)

print(f'{cnt}개의 동영상 정보를 불러왔습니다.')

18개의 동영상 정보를 불러왔습니다.


In [6]:
for (video_id, title) in video_list:
    print(f'Video ID: {video_id} / 제목: {title}')

Video ID: 4uBPe4u3k20 / 제목: 오늘도 풀 플레티넘 영상(먹이반응)
Video ID: 0aTVg1xihXY / 제목: 고정구피 풀 플레티넘 새어항 적응기
Video ID: I_EzJkM2FBI / 제목: 새로운 구피가족 알비노 풀 플레티넘!!
Video ID: -14ElrYxi2M / 제목: 랑이네 새로운 가족 1일차
Video ID: thSzZFdJf7A / 제목: 구피 입술 영상😘
Video ID: G_4Qvuxf1qs / 제목: 구피가족/ 새식구/ 우리 가족을 소개합니다/ 랑이아빠
Video ID: tcyxWaKDmJY / 제목: 2020년 10월 19일
Video ID: -GircQ4nbks / 제목: 2020년 10월 19일
Video ID: AOvBJlPGcU0 / 제목: 2020년 10월 19일
Video ID: gZpEwSw1z6Q / 제목: 2020년 10월 19일
Video ID: _BAyobrrFJg / 제목: 2020년 10월 19일
Video ID: ql3-Z5Imk-0 / 제목: 2020년 10월 19일
Video ID: zdHvTAPQ7pM / 제목: 2020년 10월 19일
Video ID: cv24KPP9NQs / 제목: 2020년 10월 19일
Video ID: T0A8r9UBpM4 / 제목: 2020년 10월 19일
Video ID: Rzu-fym-8FQ / 제목: 2020년 10월 19일
Video ID: iW8jeKG1_3E / 제목: 2020년 10월 19일
Video ID: Z6JHiTUcxlI / 제목: 2020년 10월 19일


In [7]:
# commentThreads.list() API를 호출하여 특정 비디오의 댓글 스레드(thread)를 불러오기
def get_comment_threads(youtube, video_id):
    results = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        textFormat='plainText',
        maxResults=100, # 최근 100개까지의 댓글 확인
    ).execute()

    comment_list = []
    for item in results['items']:
        comment_id = item['id']
        comment = item['snippet']['topLevelComment']
        author = comment['snippet']['authorDisplayName']
        publishedAt = comment['snippet']['publishedAt']
        text = comment['snippet']['textDisplay']
        comment_list.append((comment_id, author, publishedAt, text))

    return comment_list

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from soynlp.normalizer import *
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

file_name = "comment.xlsx"
comment = pd.read_excel(file_name)
ad =  pd.read_excel("clean_ad.xlsx")


comment = np.array(comment["comment"])
advertise = np.array(ad["comment"])
com_len = len(comment)
ad_len = len(advertise)
y_data = np.array([0]*com_len+[1]*ad_len)
comments = np.concatenate([comment,advertise])
df = pd.DataFrame(comments, columns=["comment"])
vocab_size = 2000

# 불용어 정의
df['comment'] = df['comment'].str.replace(".","")                         # .이나 _먼저 공백으로
df['comment'] = df['comment'].str.replace("_", "")
# df['comment'] = df['comment'].str.replace("[^가-힣ㅏ-ㅣㄱ-ㅎ]"," ")      # 단모음까지 남기기
df['comment'] = df['comment'].str.replace("\W"," ")                    # 특수문자 제거.
df['comment'] = df['comment'].str.replace("체널|쳬널|챼널|채녈|체녈|쳬녈|챼녈", "채널") # 채널 단어 정규화
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','데',"게","꽤", "면", "로"]


my_okt = Okt()
X_data = []
word2index={}  
bow=[]  
for a_sent in df['comment']:
#     print(a_sent)
    a_sent = emoticon_normalize(a_sent, num_repeats=2)
    a_sent = repeat_normalize(a_sent, num_repeats=2)
    a_sent_tokenized = my_okt.morphs(a_sent, stem=True)                    # 어간추출 적용 토큰화.
    a_sent_tokenized = [a_word for a_word in a_sent_tokenized if not a_word in stopwords] # 불용어 제거.
    
#     a_sent_tokenized = my_okt.nouns(a_sent)                               # 명사만.
    
    X_data.append(a_sent_tokenized)
my_tokenizer = Tokenizer(num_words= vocab_size)
my_tokenizer.fit_on_texts(X_data) 

max_len = 40

def ad_predict(a_sentence):
    a_sentence = emoticon_normalize(a_sentence, num_repeats=2)
    a_sentence = repeat_normalize(a_sentence, num_repeats=2)
    a_sentence = my_okt.morphs(a_sentence, stem=True) # 토큰화
    a_sentence = [a_word for a_word in a_sentence if not a_word in stopwords] # 불용어 처리.
    a_sentence_encoded = my_tokenizer.texts_to_sequences([a_sentence]) # 정수 인코딩.
    X_new = pad_sequences(a_sentence_encoded, maxlen = max_len)     # Padding.
    
    
    a_score = float(loaded_model.predict(X_new))                      # 예측.
#     a_score = float(my_model.predict(X_new))
    
    if a_score>0.85 :
        return True

    return False

In [22]:
video_id = '-14ElrYxi2M' # 비디오 ID
comment_list = get_comment_threads(youtube, video_id)
comment_ids = []
for (comment_id, author, publishedAt, text) in comment_list:
    print(comment_id, author, publishedAt, text)
    if ad_predict(text):
        comment_ids.append([comment_id,text])


UgwEPrLGAO85udvjsbh4AaABAg 전혜정 2021-02-24T07:45:18Z 잘보고가요~ 이름은 왜 랑이인가요?
UgzEdO8sUcPgA4UHzHp4AaABAg 전혜정 2021-02-24T07:41:05Z 어머 랑이네 가족이 많네요 몇명인가요
UgzvrXg2G1vbLqVqWCl4AaABAg 전혜정 2021-02-24T07:38:52Z 아니 님들 마카롱티비가 뭐길래 난리에요 ?
UgykjyTQpGWk9LQZx_l4AaABAg 이선구 2021-02-24T07:33:53Z 영상 진짜 재밌네요. 구독하고 갑니다~
Ugz8iy3G6uVSi24UhO14AaABAg 이선구 2021-02-24T07:33:30Z 제 체널 들어오시면 십.만원 드립니다
UgzZqLlXyHreiQx8tDt4AaABAg 제로 2021-02-17T00:23:00Z 테스트 댓글중..
Ugxo8-4zwKlj_lwSoaR4AaABAg 전혜정 2021-02-17T00:22:38Z 제 체널로 놀러오세요
Ugwn2fU_G3nHUwOTLTN4AaABAg 제로 2021-02-17T00:21:58Z 사다리무료프젝 카톢 NANO5
Ugx1aAKo_g2bnghKuAB4AaABAg 이선구 2021-02-17T00:21:44Z 랑이의 탄생을 축하드립니다!
UgxDaYCw7Nm54UKHeQF4AaABAg 제로 2021-02-17T00:21:13Z 올챙이들 너무 커여워요~
Ugxt7_NcVm18l6-3OgZ4AaABAg 랑이아빠 2021-02-16T14:16:49Z 실험용 댓글


In [21]:
banned = True # 댓글 작성자 차단 여부

# 삭제 수행 (삭제 이후에는 더 이상 API 상에서도 해당 댓글이 보이지 않음)
request = youtube.comments().setModerationStatus(
    id=[x[0] for x in comment_ids],
    moderationStatus='rejected',
    banAuthor=banned
)
request.execute()

print(f'{len(comment_ids)}개의 댓글을 삭제했습니다.')

4개의 댓글을 삭제했습니다.


In [19]:
comment_ids

[['UgxdYQpmhh_p8g2QJJZ4AaABAg',
  '제 체.널 오셔서 10먄.원 이상 벌어가셔요 진짜 있으니 해보고 속는셈 치고 한번 들러주세요 모두 행복하세용^^!ㅎ🤩🤩🤩'],
 ['UgxcdnWUb3T3leODfLZ4AaABAg', '내 쳬_널에 틱톡여신 방송★사고 난거 올려뒀어용 놀러오세용♥♥'],
 ['UgxWoTNXZ8Xl7eEg7zd4AaABAg', '마카롱티비 뭔데 여기저기 다 보이냐? 나도 다운받아봐야지'],
 ['Ugwh6Qfa7EaED1YkMLl4AaABAg', '마카롱티비 검색해봤는데 엄청나던데요? 혹시 안 보신분들 한번 보고 자요']]

In [16]:
ad_predict("사다리무료프젝 카톢 NANO5")

False

In [23]:
loaded_model.predict([2,23,231])

UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_predict_function_4420]

Function call stack:
predict_function -> predict_function -> predict_function
